In [2]:
import pandas as pd
import numpy as np


# So when do we forecast time series data?
* Generally if it DOESN'T look like white noise or random walk, YES
* if it DOES look like WN/RW,it's just not worth it

# Structure of notebook

* White Noise: Theory and Implementation
* Random Walk: Theory and Implementation
* Conclusion

# 1. White Noise

## 1.1 Theory

* essentially  a sequence of random numbers
    * like random distortions
* in model building, we look at the residuals --the difference between the predicted and actual--
    * if the residuals look like white noise, that's a good thing!
        * random fluctuations are often out of our control and beyond the scope of most models
    * when the residuals have patterns to it -> we need to do more work
    
### When do we classify a time series as white noise?
* It must satisfy these conditions
    * the average value/mean is zero
        * the fluctuations more or less even out/sum to 0
    * standard deviation is constant - it doesn't change over time
    * the correlation between time series and its lagged ersion is not signficant
        * done by investigating correlation between current time series and same times series shifted by N periods
### How do we test if a time series resembles white noise?
* Plot the times series
* Compare the mean and standard deviation over time
* Examine autocorrelation plots